In [1]:
import queue

q = queue.Queue()

In [2]:
import time
import datetime
import pymongo
from urllib.request import Request, urlopen
import Parser

class Crawler:
    
    def __init__(self, batch_limit=5000):
        
        # Connect to mongo instance
        myclient = pymongo.MongoClient("mongodb://localhost:27017/")
        db = myclient['1001']
        self.url_html_map = db['url_html_map']
        
        # Instantiate parser
        self.Parser = Parser.Parser()

        # Instantiate page_hash - maps already found urls to html
        # Used to condition on if we've visited a page
        self.page_hash = {}
        for page in self.url_html_map.find({}):
            self.page_hash[page['url']] = 1
            
        # Stopping indicator
        self.stop_search = False
        self.batch_limit = batch_limit
        
        # Var for ensuring loops dont occur
        self.urls_visited = {}
        
        self.last_request = time.time()

    def find_str(self, s, char, start_index=0):

        index = 0
        s = s[start_index+1:]
        if char in s:
            c = char[0]
            for ch in s:
                if ch == c:
                    if s[index:index+len(char)] == char:
                        return start_index + 1 + index
                index += 1
        return -1

    def request(self,url):

        req = Request(url,\
                      headers={'User-Agent': 'Mozilla/5.0'})
        html = str(urlopen(req).read())
        return html
    
    def request_db(self, url):
        
        htmls = []
        for page in self.url_html_map.find({'url': url}):
            htmls.append(page['html'])
        
        if len(htmls) == 1:
            return htmls[0]
        if len(htmls) == 0:
            return False
        if len(htmls) > 1:
            return htmls[0]
        
    def parse(self, url, html): 

        self.Parser.parse(url, html)
    
    def crawl(self, url):
        
        # Insert start url into queue
        self.q = queue.Queue()
        self.q.put(url)
        
        # Iterate until queue is empty
        while not self.q.empty():
            
            # Pop from queue (fifo)
            url = self.q.get()
        
            # Check for stopping conditions
            if self.Parser.tracklist_num == self.batch_limit:
                print('STOPPING SEARCH')
                self.stop_search = True
            print(url, self.page_hash.get(url, 0))
            if (self.stop_search):
                return

            # If in db then pull html
            if (self.page_hash.get(url, 0) == True):

                print('requesting db')
                html = self.request_db(url)
                if html == False:
                    print('MORE THAN ONE FOUND FOR:')
                    print(url)
                    continue

            # If not in db then http request
            if (self.page_hash.get(url, 0) == False):

                print('requesting http')
                # Only sleep if gap not enough
                if time.time() - self.last_request < 5:
                    time.sleep(5 - (time.time() - self.last_request))

                # Make http request
                try:
                    html = self.request(url)
                except:
                    print('REQUEST FAILED')
                    continue
                self.last_request = time.time()
                
                # If http requested then parse and extract necessary data 
                if ('/tracklist/' in url):
                    self.parse(url, html)
                    self.page_hash[url] = 1

            print('finding links')
            # Iterate over links found in html
            index = 0
            while self.find_str(html, 'a href="', index) != -1:

                # Extract url
                index = self.find_str(html, 'a href="', index)
                url_chunk = html[index:].split('"')[1]

                # Make sure it is either a referenced tracklist or 1001 page
                # and not already reached by search
                if ('/tracklist/' in url_chunk) and\
                   ('http' not in url_chunk) and\
                   ('#tlp' not in url_chunk) and\
                   (self.urls_visited.get(url_chunk, 0) == False): # <- compare short address to store from search

                    self.urls_visited[url_chunk] = 1 # only shortened address stored
                    new_page = 'https://www.1001tracklists.com' + url_chunk
                    self.q.put(new_page)

                elif ('/dj/' in url_chunk) and\
                     ('http' not in url_chunk) and\
                     ('#tlp' not in url_chunk) and\
                     (self.urls_visited.get(url_chunk, 0) == False):

                    self.urls_visited[url_chunk] = 1
                    new_page = 'https://www.1001tracklists.com' + url_chunk
                    self.q.put(new_page)

                elif ('www.1001tracklists.com' in url_chunk) and\
                     ('#tlp' not in url_chunk) and\
                     ('.xml' not in url_chunk) and\
                     (self.urls_visited.get(url_chunk, 0) == False):

                    self.urls_visited[url_chunk] = 1
                    self.q.put(url_chunk)

            # Cache url-html map
            self.page_hash[url] = html
      

In [3]:
crawler = Crawler(batch_limit=100000)

In [ ]:
crawler.crawl('https://www.1001tracklists.com/')